# Assignment 3  
## Applied Machine Learning

Andrew Chan 
EBE869

In [188]:
import pandas as pd
import math
import numpy as np

# Locate and load the data file
df = pd.read_csv('master.csv')

# Sanity check
print(f'N rows={len(df)}, M columns={len(df.columns)}')
df.head()

N rows=27820, M columns=12


,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [189]:
df.columns.values

array(['country', 'year', 'sex', 'age', 'suicides_no', 'population',
       'suicides/100k pop', 'country-year', 'HDI for year',
       ' gdp_for_year ($) ', 'gdp_per_capita ($)', 'generation'],
      dtype=object)

# 1. [20 pts] Due to the severity of this real-world crisis, what information would be the most important one to "machine learn"? Can it be learned?

The most important target variable to machine learn is `suicides_no`. To attempt a machine learning solution we will turn this into a binary classification problem of `low suicide rate` versus `high suicide rate`. Framing the problem this way will allow us to train a classifier. 

# 2. [20 pts] Explain in detail how one should set up the problem? Would it be a regression or a classification problem? Any unsupervised approach to see some patterns exist or not?

We will frame this as a classification problem with `low suicide rate` and `high suicide rate`. 

# 3. [20 pts] What should be the dependent variable?

The dependent variable will be generated from the `suicides_no`.

In [190]:
suicides_mean = df["suicides_no"].mean()
suicides_mean

242.57440690150972

We will create a modify `suicides_no` as follows:  
* All values larger than or equal to `suicides_mean` will be labelled with `high suicide rate` 
* All values smaller than `suicides_mean` will be labelled with `low suicide rate` 

In [191]:
def isHigh(x, mean_val):
    if x > mean_val:
        return True
    else:
        return False

In [192]:
df['suicides_no'] = df['suicides_no'].apply(isHigh, args=[suicides_mean])

In [193]:
df['suicides_no'].value_counts()

False    23109
True      4711
Name: suicides_no, dtype: int64

# 4. [20 pts] Rank the variables to find some strong correlations between the independent variables and the dependent variable you decided

In [195]:
df_correlation = df.corr()

In [197]:
df_correlation.sort_values(by='suicides_no',ascending=False)

,year,suicides_no,population,suicides/100k pop,HDI for year,gdp_per_capita ($)
suicides_no,-0.014454,1.000000,0.540471,0.328492,0.126847,0.054889
population,0.008850,0.540471,1.000000,0.008285,0.102943,0.081510
suicides/100k pop,-0.039037,0.328492,0.008285,1.000000,0.074279,0.001785
HDI for year,0.366786,0.126847,0.102943,0.074279,1.000000,0.771228
gdp_per_capita ($),0.339134,0.054889,0.081510,0.001785,0.771228,1.000000
year,1.000000,-0.014454,0.008850,-0.039037,0.366786,0.339134


# 5. [20 pts] Pre-process the dataset and list the major features you want to use. 
Note that not all features are crucial. For example, country-year variable is a derived feature and for a classifier it would not be necessary to include the year, the country and the country-year together. In fact, one must avoid adding a derived feature and the original at the same time.

List the independent features you want to use.

## Preprocessing Steps

## Mean imputation

In [204]:
df.fillna(df.mean())

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,False,312900,6.71,Albania1987,0.776601,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,False,308000,5.19,Albania1987,0.776601,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,False,289700,4.83,Albania1987,0.776601,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,False,21800,4.59,Albania1987,0.776601,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,False,274300,3.28,Albania1987,0.776601,"2,156,624,900",796,Boomers
...,...,...,...,...,...,...,...,...,...,...,...,...
27815,Uzbekistan,2014,female,35-54 years,False,3620833,2.96,Uzbekistan2014,0.675000,"63,067,077,179",2309,Generation X
27816,Uzbekistan,2014,female,75+ years,False,348465,2.58,Uzbekistan2014,0.675000,"63,067,077,179",2309,Silent
27817,Uzbekistan,2014,male,5-14 years,False,2762158,2.17,Uzbekistan2014,0.675000,"63,067,077,179",2309,Generation Z
27818,Uzbekistan,2014,female,5-14 years,False,2631600,1.67,Uzbekistan2014,0.675000,"63,067,077,179",2309,Generation Z


## List of features selected

# 6. [20 pts bonus] Devise a classification problem and present a prototype model.
Note that we will continue to this problem in the following modules.